In [ ]:
# For Google Colab
%%time
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

In [ ]:
# Added 'mamba' in the conda_env.yaml file
!CONDA_CHANNEL_PRIORITY=disabled PIP_NO_DEPS=1 mamba env update -n base -f conda_env.yaml

In [1]:
import os, re, subprocess
import molviewspec as mvs
import gradio as gr

from predict import get_cif, predict

/home/fnerin/miniconda3/envs/allopockets/lib/python3.11/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python311.Release 2025.24+release.8e1e5e54f047b0833dcf760a5cd5d3ce94d63938 2025-06-06T09:20:57] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.ubuntu.cxx11thread.ser

In [2]:
os.makedirs("gradio", exist_ok=True)

In [3]:
# predictions = predict('6t4k', 'A', "gradio/0w6gw9h3svrm", email="nerinfonzf98@univie.ac.at"),

In [4]:
mvs_pre, mvs_post = mvs.molstar_widgets.VIEWER_TEMPLATE.replace("{{version}}", "latest").replace("{{format}}", "mvsx").split("{{state}}")

In [5]:
def get_predictions(pdb_id, chains, path):
    subprocess.run(f"cp gradio/pphf19lz5bb/6t4k_1.*m {path}/", shell=True)
    os.makedirs(f"{path}/features/6t4k", exist_ok=True)
    subprocess.run(f"cp gradio/pphf19lz5bb/features/6t4k/PyRosettaF.pkl {path}/features/6t4k/", shell=True)
    return gr.update(
        value=predict(pdb_id.lower(), chains, path, email="nerinfonzf98@univie.ac.at").reset_index(names="Pocket"),
        visible=True
    )

In [6]:
def get_mvsData(cif):
    builder = mvs.create_builder()
    (
        builder
        .download(url="cif")
        .parse(format="mmcif")
        .model_structure()
        .component()
        .representation()
    )
    return (
        mvs.MVSX(
            data=builder.get_state(),
            assets={'cif': cif.cif.text.encode()}
        )
        .molstar_html()
        .removeprefix(mvs_pre).removesuffix(mvs_post)
    )[1:-1] # delete initial and trailiing #

In [10]:
demo.close()

def get_path(request: gr.Request):
    path = f"gradio/{request.session_hash}"
    os.makedirs(path, exist_ok=True)
    return path


viewer = """
(mvs_data) => {
    const mvsData = mvs_data;

    molstar.Viewer
        .create('viewer1', { layoutIsExpanded: false, layoutShowControls: false })
        .then(viewer => viewer.loadMvsData(mvsData, 'mvsx'));
}
"""

def process_pdb_id(pdb_id, path):
    if re.fullmatch(r"[0-9a-zA-Z]{4}", pdb_id):
        try:
            cif = get_cif(pdb_id.lower(), path=path)
        except:
            gr.Warning(f"{pdb_id.lower()} PDB ID could not be retrieved")
            return
            
        chains = cif.residues.query(f"label_entity_id in {cif._protein_entities}").label_asym_id.unique().tolist()
        mvsx = get_mvsData(cif)
        
        return (
            gr.update(value=mvsx),
            gr.update(choices=chains, visible=True, interactive=True),
            gr.update(visible=True),
            gr.update(value=True),
        )
    return (
        gr.update(value=""), gr.update(choices=[], visible=False),
        gr.update(visible=False),
        gr.update(value=False),
    )



with gr.Blocks(
    head="""
        <script src="https://cdn.jsdelivr.net/npm/molstar@latest/build/viewer/molstar.js"></script>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/molstar@latest/build/viewer/molstar.css" />
    """
) as demo:
    with gr.Row():
        with gr.Column(min_width=375):
            gr.Markdown("# AlloPockets")
            # with gr.Row():
            pdb_id = gr.Textbox(label="Enter a valid PDB ID:", placeholder="e.g, 6t4k")
            # view_btn = gr.Button("View PDB")
            # with gr.Row():
            chain_id = gr.CheckboxGroup(label="Select the desired chain ID(s):", info="label_asym_id", choices=[], visible=False)
            predict_btn = gr.Button("Predict allosteric pockets", visible=False, variant="primary")
            predictions = gr.Dataframe(visible=False, show_copy_button=True, max_height=175, show_row_numbers=True)
            selected = gr.Textbox()
    
        with gr.Column(scale=100):#scale=10
            gr.HTML('<div id="viewer1" style="width: 100%; height: 750px;"></div>')

    # Fake elements
    path = gr.State()#gr.Text(interactive=False, visible=False)
    mvsx = gr.Text(interactive=False, visible=False)
    view_pdb = gr.Checkbox(interactive=False, visible=False)

    demo.load(
        fn = get_path,
        outputs = path
    )

    # Update everything when PDB ID is changed
    pdb_id.change(
        fn=process_pdb_id,
        inputs=[pdb_id, path],
        outputs=[
            # Updated
            mvsx, chain_id, 
            # Visibility
            predict_btn,
            # Toggle
            view_pdb
        ],
    )
    view_pdb.change(
        fn=None, inputs=mvsx, js=viewer
    )

    predict_btn.click(fn=get_predictions, inputs=[pdb_id, chain_id, path], outputs=[predictions])
    
    def show_prediction(evt: gr.SelectData): return evt.value
    predictions.select(show_prediction, outputs=selected)

demo.launch(server_port=8998)

Closing server running on port: 8998


2025-07-09 13:12:00,368 - httpx - INFO - HTTP Request: GET http://127.0.0.1:8998/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-07-09 13:12:00,375 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:8998/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:8998
* To create a public link, set `share=True` in `launch()`.


2025-07-09 13:12:01,087 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 


  0%|          | 0/9 [00:00<?, ?it/s]

In [11]:
import pandas as pd

In [14]:
pd.DataFrame([1,1,1,1,1,1,1,11]).reset_index(names="Pocket rank")

,Pocket rank,0
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,11


In [3]:
cif = get_cif("6t4k", ".")

In [4]:
cif.cif.text

"data_6T4K\n#\n_entry.id 6T4K\n#\n_citation.abstract ?\n_citation.abstract_id_CAS ?\n_citation.book_id_ISBN ?\n_citation.book_publisher ?\n_citation.book_publisher_city ?\n_citation.book_title ?\n_citation.coordinate_linkage ?\n_citation.country US\n_citation.database_id_Medline ?\n_citation.details ?\n_citation.id primary\n_citation.journal_abbrev 'Proc. Natl. Acad. Sci. U.S.A.'\n_citation.journal_id_ASTM PNASA6\n_citation.journal_id_CSD 0040\n_citation.journal_id_ISSN 1091-6490\n_citation.journal_full ?\n_citation.journal_issue ?\n_citation.journal_volume 118\n_citation.language ?\n_citation.page_first ?\n_citation.page_last ?\n_citation.title 'Cooperativity between the orthosteric and allosteric ligand binding sites of ROR gamma t.'\n_citation.year 2021\n_citation.database_id_CSD ?\n_citation.pdbx_database_id_DOI 10.1073/pnas.2021287118\n_citation.pdbx_database_id_PubMed 33536342\n_citation.unpublished_flag ?\n#\nloop_\n_citation_author.citation_id\n_citation_author.name\n_citation_